In [1]:
from define import qcode as qc
import numpy as np
from define.QECCLfid import uncorrectable as uc
from define import globalvars as gv

In [67]:
def uncorr_l2(pauli_probs,qcode):
    # compute coset probabilities
    nlogs = 4**qcode.K
    nstabs = 2**(qcode.N - qcode.K)
    nblocksl1 = qcode.N
    coset_probs = np.zeros(nlogs,dtype=np.double)
    for i in range(nlogs):
        inds = i*((2**(qcode.N-qcode.K))**2) + range(nstabs)*(2**(qcode.N-qcode.K))
        coset_probs[i] = np.sum(pauli_probs[inds])
    # get level-2 contribution
    # Note not including all Identity correctable because it is accounted for in the l1 case
    correctable_prob_l2 = np.sum(list(map(lambda x: np.prod([coset_probs[i] for i in x]),qcode.Paulis_correctable[1:])))
    # Get level-1 contribution
    correctable_prob_l1 = sum([pauli_probs[p] for p in qcode.PauliCorrectableIndices])**(nblocksl1)
    print("Level 1 correctable : {}".format(correctable_prob_l1))
    print("Level 2 correctable : {}".format(correctable_prob_l2))
    
    return 1 - (correctable_prob_l1 + correctable_prob_l2)
    
            
            
            
        
    

In [65]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)
qc.populate_symplectic(qcode)
uc.ComputeCorrectableIndices(qcode)
correctable_set = qcode.Paulis_correctable

In [71]:
probs = np.random.random(4**7)
probs[0] = 10000
probs/= np.sum(probs)
print(probs[0])
uncorr_l2(probs,qcode)

0.5488505920872389
Level 1 correctable : 0.05633348270120015
Level 2 correctable : 0.0010214050818171502


0.9426451122169827

In [47]:
qcode.PauliCorrectableIndices

[0,
 64,
 128,
 192,
 256,
 320,
 384,
 448,
 512,
 576,
 640,
 704,
 768,
 832,
 896,
 960,
 1024,
 1088,
 1152,
 1216,
 1280,
 1344,
 1408,
 1472,
 1536,
 1600,
 1664,
 1728,
 1792,
 1856,
 1920,
 1984,
 2048,
 2112,
 2176,
 2240,
 2304,
 2368,
 2432,
 2496,
 2560,
 2624,
 2688,
 2752,
 2816,
 2880,
 2944,
 3008,
 3072,
 3136,
 3200,
 3264,
 3328,
 3392,
 3456,
 3520,
 3584,
 3648,
 3712,
 3776,
 3840,
 3904,
 3968,
 4032,
 8193,
 8257,
 8321,
 8385,
 8449,
 8513,
 8577,
 8641,
 8705,
 8769,
 8833,
 8897,
 8961,
 9025,
 9089,
 9153,
 9217,
 9281,
 9345,
 9409,
 9473,
 9537,
 9601,
 9665,
 9729,
 9793,
 9857,
 9921,
 9985,
 10049,
 10113,
 10177,
 10241,
 10305,
 10369,
 10433,
 10497,
 10561,
 10625,
 10689,
 10753,
 10817,
 10881,
 10945,
 11009,
 11073,
 11137,
 11201,
 11265,
 11329,
 11393,
 11457,
 11521,
 11585,
 11649,
 11713,
 11777,
 11841,
 11905,
 11969,
 12033,
 12097,
 12161,
 12225,
 8194,
 8258,
 8322,
 8386,
 8450,
 8514,
 8578,
 8642,
 8706,
 8770,
 8834,
 8898,
 896